In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Usage</a></span></li></ul></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Imports-/-load-data" data-toc-modified-id="Imports-/-load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports / load data</a></span></li><li><span><a href="#Main-calculation" data-toc-modified-id="Main-calculation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Main calculation</a></span></li></ul></div>

# Overview

Given
  - a filepath $d$ to an `.npy` file defining $p(V|C)$ a distribution on orthographic wordforms $V$ given (orthographic) $n$-gram context $c \in C$
  - a filepath $w$ to an `.npz` file defining $p(W|V)$ a distribution on full segmental wordforms $W$ given an orthographic wordform $v \in V$
  - a filepath $m$ to an `.npy` file defining $p(W|C)$ a distribution on full segmental wordforms $W$ given an (orthographic) $n$-gram context $c \in C$
  - an output filepath prefix $o$

this notebook calculates $p(V|W,C)$, i.e.

$$p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$$

and exports it to $o$`.npy`.

## Requirements

 - `numpy`
 - the `pydata` `sparse` package

## Usage

In [ ]:
#FIXME

# Parameters

In [2]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

In [3]:
from boilerplate import *

In [4]:
# Parameters

d = ''
d = 'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts_projected_LTR_Buckeye.pV_C.npy'

w = ''
w = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz'

m = ''
m = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy'

o = ''
o = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_WC'

In [5]:
output_dir = path.dirname(o)
ensure_dir_exists(output_dir)

# Imports / load data

In [8]:
import sparse

In [11]:
pV_C = np.load(d)
pV_C.shape
pV_C.dtype
pV_C.nbytes / 1e9

(6574, 16443)

dtype('float64')

0.864770256

In [12]:
pW_V = sparse.load_npz(w)
pW_V.shape
pW_V.dtype
pW_V.nbytes / 1e9
pW_V.density

(6403, 6574)

dtype('float64')

0.000157776

0.00015617679212868969

In [13]:
pW_C = np.load(m)
pW_C.shape
pW_C.dtype
pW_C.nbytes / 1e9

(6403, 16443)

dtype('float64')

0.842276232

# Main calculation

$$p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$$

Let
 - $d = p(V|C)$
 - $w = p(W|V)$
 - $m = p(W|C)$
 - $o = p(V|W,C)$

$o_{i,j,k} = \frac{w_{j,i} d_{i,k}}{m_{j,k}}$

In [ ]:
pV_WC = 